In [2]:
import ccxt
import dontshare_config as dc
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [5]:
# Créer une instance du client Mexc
mexc = ccxt.mexc({
    'apiKey': dc.API_KEY_MEXC,# Clé API publique
    'secret': dc.API_SECRET_MEXC, # Clé API privée
    'enableRateLimit': True,  # Activer la limite de taux si nécessaire
})

mexc_futures = ccxt.mexc({
    'apiKey': dc.API_KEY_MEXC,
    'secret': dc.API_SECRET_MEXC,
    'enableRateLimit': True,  # Activer la limite de taux si nécessaire
    "options": {'defaultType': 'swap' } # Défini comppte FUTURES
})

binance_testnet = ccxt.binance({
    'apiKey': dc.API_KEY_BINANCE,
    'secret': dc.API_SECRET_BINANCE,
    'enableRateLimit': True,  # Activer la limite de taux si nécessaire
})

coinbase_testnet = ccxt.coinbase({
    'apiKey': dc.API_KEY_COINBASE,
    'secret': dc.API_SECRET_COINBASE,
    'enableRateLimit': True,  # Activer la limite de taux si nécessaire
})

exchange = coinbase_testnet # Choix de l'exchange sur lequel les opérations sont éffectuées
#exchange.set_sandbox_mode(True)  # enable sandbox mode

Récupération du compte


In [6]:
try:
    balance = exchange.fetch_balance()

    # Créer un DataFrame pandas pour stocker les informations du compte
    account_info = {
        'Currency': [],
        'Total': [],
        'Free': [],
        'Used': [],
    }

    for currency, data in balance['total'].items():
        account_info['Currency'].append(currency)
        account_info['Total'].append(data)
        account_info['Free'].append(balance['free'][currency])
        account_info['Used'].append(balance['used'][currency])

    df_account = pd.DataFrame(account_info)

    # Afficher le DataFrame
    print(df_account)

except ccxt.NetworkError as e:
    print('Problème de connexion : ', type(e).__name__, str(e))
except ccxt.ExchangeError as e:
    print('Erreur d\'échange : ', type(e).__name__, str(e))
except Exception as e:
    print('Une erreur s\'est produite : ', type(e).__name__, str(e))

Erreur d'échange :  AuthenticationError coinbase GET https://api.coinbase.com/api/v3/brokerage/products 401 Unauthorized Unauthorized


Récupération des dernières bougies


In [7]:
# Paramètres pour la récupération des bougies
symbol = 'BTC/USDT'
timeframe = "1m"

# Récupérer les bougies
try:
    candles = exchange.fetch_ohlcv(symbol, timeframe)
    
    # Créer une liste pour stocker les données des bougies
    candle_data = []

    for candle in candles:
        timestamp, open_, high, low, close, volume = candle
        dt_object = datetime.utcfromtimestamp(timestamp / 1000)  # Convertir le timestamp en objet datetime
        candle_data.append([dt_object, open_, high, low, close, volume])

    # Créer un DataFrame pandas
    columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(candle_data, columns=columns)

    # Afficher le DataFrame
    print(df)

except ccxt.NetworkError as e:
    print('Problème de connexion : ', type(e).__name__, str(e))
except ccxt.ExchangeError as e:
    print('Erreur d\'échange : ', type(e).__name__, str(e))
except Exception as e:
    print('Une erreur s\'est produite : ', type(e).__name__, str(e))


Erreur d'échange :  AuthenticationError coinbase GET https://api.coinbase.com/api/v3/brokerage/products 401 Unauthorized Unauthorized


In [8]:
def place_order(symbol, direction, leverage, stop_loss, take_profit, investment_value, limit_price):

    # Direction : long or short
    # 

    # Order parameters
    side = 'buy' if direction == 'long' else 'sell'
    leverage = leverage

    # Create the order
    order_params = {
        'symbol': symbol,
        'side': side,
        'type': 'limit',  # You can adjust the order type according to your needs
        'amount': investment_value,
        'stopPrice': stop_loss,
        'takeProfitPrice': take_profit,
        'price': limit_price,  # Specify the limit price
    }

    try:
        # Place the order
        order = exchange.create_order(**order_params)
        print('Order placed successfully:', order)

    except ccxt.NetworkError as e:
        print('Connection problem: ', type(e).__name__, str(e))
    except ccxt.ExchangeError as e:
        print('Exchange error: ', type(e).__name__, str(e))
    except Exception as e:
        print('An error occurred: ', type(e).__name__, str(e))

# Example of usage for a limit order
place_order('BTC/USDT', 'long', 10, 50000, 60000, 1, 55000)



An error occurred:  TypeError coinbase.create_order() got an unexpected keyword argument 'stopPrice'


In [13]:
symbol = 'ETH/BTC'
type = 'limit'  # or 'market'
side = 'buy'
amount = 123.45  # your amount
price = 115.321  # your price
params = {
    'stopLoss': {
        'type': 'limit',  # or 'market', this field is not necessary if limit price is specified
        'price': 100.33,  # limit price for a limit stop loss order
        'triggerPrice': 101.25,
    },
    'takeProfit': {
        'type': 'market',  # or 'limit', this field is not necessary if limit price is specified
        # no limit price for a market take profit order
        # 'price': 160.33,  # this field is not necessary for a market take profit order
        'triggerPrice': 150.75,
    }
}
order = exchange.create_order (symbol, type, side, amount, price, params)

ExchangeError: mexc {"code":10004,"msg":"symbol not support api"}

In [64]:
exchange.create_order ('SOL/USDC', 'limit', 'buy', 0.01, 800)

{'id': 'C01__352873160120553473',
 'clientOrderId': None,
 'timestamp': 1699913753305,
 'datetime': '2023-11-13T22:15:53.305Z',
 'lastTradeTimestamp': None,
 'status': None,
 'symbol': 'SOL/USDC',
 'type': 'limit',
 'timeInForce': None,
 'side': 'buy',
 'price': 800,
 'stopPrice': None,
 'triggerPrice': None,
 'average': None,
 'amount': 0.01,
 'cost': None,
 'filled': None,
 'remaining': None,
 'fee': None,
 'trades': [],
 'info': {'symbol': 'SOLUSDC',
  'orderId': 'C01__352873160120553473',
  'orderListId': '-1',
  'price': '800',
  'origQty': '0.01',
  'type': 'LIMIT',
  'side': 'BUY',
  'transactTime': '1699913753305'},
 'fees': [],
 'lastUpdateTimestamp': None,
 'postOnly': None,
 'reduceOnly': None,
 'takeProfitPrice': None,
 'stopLossPrice': None}

In [71]:
exchange.create_order ('SOL/USDC', 'limit', 'buy', 0.6, 10)

ExchangeError: mexc {"code":10004,"msg":"symbol not support api"}